## Creating Word Embedding Models for 4 decades

Computational Literature Review

Creator: Jaren Haber, PhD, Nancy Xu

Date created: February 15, 2022

Date last modified: April 11, 2022

This notebook preprocesses training texts, and creates word2vec embedding models for 4 decades (1970-1979,1980-1989,1990-1999,2000-2015)

## Load datasets

In [1]:

import pickle
import re
from tqdm import tqdm

import os
import random as rand

import numpy as np
import pandas as pd
# from clean_text import stopwords_make, punctstr_make, unicode_make, apache_tokenize, clean_sentence_apache 

import matplotlib.pyplot as plt

import numpy as np
import re
import random

from collections import Counter


def open_test_data(path):
    return open(path, 'rb')
with open_test_data('/home/jovyan/work/tlu_storage/training_cultural_preprocessed_100321.pkl') as f:
    cult = pickle.load(f)

with open_test_data('/home/jovyan/work/tlu_storage/training_demographic_preprocessed_100321.pkl') as f:
    demog = pickle.load(f)

with open_test_data('/home/jovyan/work/tlu_storage/training_orgs_preprocessed_100321.pkl') as f:
    orgs = pickle.load(f)

with open_test_data('/home/jovyan/work/tlu_storage/training_relational_preprocessed_100321.pkl') as f:
    rela = pickle.load(f)
    

In [ ]:
def modify_jstor_id(x, col_name, https = False):
    '''
    modify jstor id to get the link in the form of www.jstor.org/stable/23057056
    '''
    
    good_parts = []
    if not https:
        for ii in x[col_name]:
            try: 
                good_parts.append(ii.split('http://')[1])
            except:
                good_parts.append(ii)
    else:
        for ii in x[col_name]:
            try: 
                good_parts.append(ii.split('https://')[1])
            except:
                good_parts.append(ii)
        
    return good_parts

In [ ]:
# modify jstor id's

## combine the data for the correct article dates
dates = pd.read_csv('./sample/parts-1-3-metadata.csv')
date2 = pd.read_csv('./sample/part-4-metadata.csv')


date2.id = modify_jstor_id(date2, 'id')


dates.id = modify_jstor_id(dates,'id')

In [ ]:
combo = pd.concat([dates, date2])

In [ ]:
root = '/home/jovyan/work/'
meta_fp = root + '/dictionary_methods/code/metadata_combined.h5' 

df_meta = pd.read_hdf(meta_fp)
df_meta.reset_index(drop=False, inplace=True) # extract file name from index

# For merging purposes, get ID alone from file name, e.g. 'journal-article-10.2307_2065002' -> '10.2307_2065002'
df_meta['edited_filename'] = df_meta['file_name'].apply(lambda x: x[16:]) 
df_meta = df_meta[["edited_filename", "article_name", "jstor_url", "abstract", "journal_title", "given_names", "primary_subject", "year", "type"]] # keep only relevant columns

df_meta['id'] =  modify_jstor_id(df_meta,'jstor_url', True)


In [ ]:
m = df_meta.merge(combo, on = 'id')

In [ ]:
mm_cult = cult.merge(m, on = 'edited_filename', how = 'left')[['text', 'cultural_score', 'edited_filename', 'journal_title', 'publicationYear']]
mm_cult = mm_cult[~mm_cult['publicationYear'].isna()]

In [ ]:
mm_orgs= orgs.merge(m, on = 'edited_filename', how = 'left')[['text', 'orgs_score', 'edited_filename', 'journal_title', 'publicationYear']]
mm_orgs = mm_orgs[~mm_orgs['publicationYear'].isna()]


In [ ]:
mm_rela= rela.merge(m, on = 'edited_filename', how = 'left')[['text', 'relational_score', 'edited_filename', 'journal_title', 'publicationYear']]
mm_rela = mm_rela[~mm_rela['publicationYear'].isna()]


In [ ]:
mm_demog= demog.merge(m, on = 'edited_filename', how = 'left')[['text', 'demographic_score', 'edited_filename', 'journal_title', 'publicationYear']]
mm_demog= mm_demog[~mm_demog['publicationYear'].isna()]


## Remove HTML

In [ ]:
import itertools
full_text = []

for i in mm_cult['text']:
    joined = list(itertools.chain(*i))
    full_text.append(" ".join(joined))

full_text_demog = []
for i in mm_demog['text']:
    joined = list(itertools.chain(*i))
    full_text_demog.append(" ".join(joined))
    
full_text_orgs = []
for j in mm_orgs['text']:
    joined = list(itertools.chain(*j))
    full_text_orgs.append(" ".join(joined))

full_text_rela = []
for j in mm_rela['text']:
    joined = list(itertools.chain(*j))
    full_text_rela.append(" ".join(joined))
    
mm_cult['full_text'] = full_text
mm_demog['full_text'] = full_text_demog
mm_orgs['full_text'] = full_text_orgs
mm_rela['full_text'] = full_text_rela

def remove_tags(article):
    article = re.sub('<plain_text> <page sequence="1">', '', article)
    article = re.sub(r'</page>(\<.*?\>)', ' \n ', article)
    # xml tags
    article = re.sub(r'<.*?>', '', article)
    article = re.sub(r'<body.*\n\s*.*\s*.*>', '', article)
    return article

tags_removed = [remove_tags(art) for art in mm_cult['full_text']]
tags_removed_demog = [remove_tags(art) for art in mm_demog['full_text']]
tags_removed_org = [remove_tags(art) for art in mm_orgs['full_text']]
tags_removed_rela = [remove_tags(art) for art in mm_rela['full_text']]
mm_cult['text_no_tags'] = tags_removed
mm_demog['text_no_tags'] = tags_removed_demog
mm_orgs['text_no_tags'] = tags_removed_org
mm_rela['text_no_tags'] = tags_removed_rela

## Remove stop words, punctuations, lemmatize

In [ ]:
# Prep dictionaries of English words
from nltk.corpus import words # Dictionary of 236K English words from NLTK
english_nltk = set(words.words()) # Make callable
english_long = set() # Dictionary of 467K English words from https://github.com/dwyl/english-words
# fname =  "english_words.txt" # Set file path to long english dictionary
# with open(fname, "r") as f:
#     for word in f:
#         english_long.add(word.strip())
        
def stopwords_make(vocab_path_old = "", extend_stopwords = False):
    """Create stopwords list. 
    If extend_stopwords is True, create larger stopword list by joining sklearn list to NLTK list."""
                                                     
    stop_word_list = list(set(stopwords.words("english"))) # list of english stopwords

    # Add dates to stopwords
    for i in range(1,13):
        stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
    for i in range(1,13):
        stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
    for i in range(1, 2100):
        stop_word_list.append(str(i))

    # Add other common stopwords
    stop_word_list.append('00') 
    stop_word_list.extend(['mr', 'mrs', 'sa', 'fax', 'email', 'phone', 'am', 'pm', 'org', 'com', 
                           'Menu', 'Contact Us', 'Facebook', 'Calendar', 'Lunch', 'Breakfast', 
                           'facebook', 'FAQs', 'FAQ', 'faq', 'faqs']) # web stopwords
    stop_word_list.extend(['el', 'en', 'la', 'los', 'para', 'las', 'san']) # Spanish stopwords
    stop_word_list.extend(['angeles', 'diego', 'harlem', 'bronx', 'austin', 'antonio']) # cities with many charter schools

    # Add state names & abbreviations (both uppercase and lowercase) to stopwords
    states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 
              'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 
              'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 
              'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 
              'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WI', 'WV', 'WY', 
              'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
              'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 
              'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 
              'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
              'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
              'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
              'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 
              'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 
              'Vermont', 'Virginia', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming' 
              'carolina', 'columbia', 'dakota', 'hampshire', 'mexico', 'rhode', 'york']
    for state in states:
        stop_word_list.append(state)
    for state in [state.lower() for state in states]:
        stop_word_list.append(state)
        
    # Add even more stop words:
    if extend_stopwords == True:
        stop_word_list = text.ENGLISH_STOP_WORDS.union(stop_word_list)
        
    # If path to old vocab not specified, skip last step and return stop word list thus far
    if vocab_path_old == "":
        return stop_word_list

    # Add to stopwords useless and hard-to-formalize words/chars from first chunk of previous model vocab (e.g., a3d0, \fs19)
    # First create whitelist of useful terms probably in that list, explicitly exclude from junk words list both these and words with underscores (common phrases)
    whitelist = ["Pre-K", "pre-k", "pre-K", "preK", "prek", 
                 "1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th", "9th", "10th", "11th", "12th", 
                 "1st-grade", "2nd-grade", "3rd-grade", "4th-grade", "5th-grade", "6th-grade", 
                 "7th-grade", "8th-grade", "9th-grade", "10th-grade", "11th-grade", "12th-grade", 
                 "1st-grader", "2nd-grader", "3rd-grader", "4th-grader", "5th-grader", "6th-grader", 
                 "7th-grader", "8th-grader", "9th-grader", "10th-grader", "11th-grader", "12th-grader", 
                 "1stgrade", "2ndgrade", "3rdgrade", "4thgrade", "5thgrade", "6thgrade", 
                 "7thgrade", "8thgrade", "9thgrade", "10thgrade", "11thgrade", "12thgrade", 
                 "1stgrader", "2ndgrader", "3rdgrader", "4thgrader", "5thgrader", "6thgrader", 
                 "7thgrader", "8thgrader", "9thgrader", "10thgrader", "11thgrader", "12thgrader"]
    with open(vocab_path_old) as f: # Load vocab from previous model
        junk_words = f.read().splitlines() 
    junk_words = [word for word in junk_words[:8511] if ((not "_" in word) 
                                                         and (not any(term in word for term in whitelist)))]
    stop_word_list.extend(junk_words)
                                                     
    return stop_word_list

In [ ]:
stop_words = stopwords_make(vocab_path_old = "", extend_stopwords = False)

In [ ]:
import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['one', 'two', 'three', 'amp', 'may', 'can', 'new', 'also', 'and'])

import string
import re
import nltk

def word_process(tt):
    """
    helper function to lower text, remove stop words, numbers, and empty 
    """
    
    tt = tt.lower()
    
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~=\n'''
    # Removing punctuations in string 
    # Using loop + punctuation string 


    for ele in tt:  
        if ele in punc:  
            tt = tt.replace(ele, " ")  

    # read tokens
    tokens = tt.split()
    lst = [token.translate(punc).lower() for token in tokens ]
    
    #remove stop words
    filtered = []
    for i in lst:
        if i not in stop_words:
            filtered.append(i)
    
    # removing singular numbers and singular letters
    pattern = '[0-9]'
    filtered = [re.sub(pattern, '', i) for i in filtered] 
    new = []
    for inp in filtered:
        new.append(' '.join( [w for w in inp.split() if len(w)>1] ))
        
    # filter out empty strings 
    new = [i for i in new if i] 

    dt = [d.split() for d in new]
    
    return dt


In [ ]:
mm_cult['text_no_tags'] = tags_removed
mm_demog['text_no_tags'] = tags_removed_demog
mm_orgs['text_no_tags'] = tags_removed_org
mm_rela['text_no_tags'] = tags_removed_rela

mm_cult['processed'] = [word_process(i) for i in mm_cult['text_no_tags']]
mm_demog['processed'] = [word_process(i) for i in mm_demog['text_no_tags']]
mm_orgs['processed'] = [word_process(i) for i in mm_orgs['text_no_tags']]
mm_rela['processed'] = [word_process(i) for i in mm_rela['text_no_tags']]
mm_rela['processed'] = [sum(i, []) for i in mm_rela['processed']]
mm_orgs['processed'] = [sum(i, []) for i in mm_orgs['processed']]
mm_demog['processed'] = [sum(i, []) for i in mm_demog['processed']]
mm_cult['processed'] = [sum(i, []) for i in mm_cult['processed']]
combo_train_df = pd.concat([mm_rela, mm_orgs, mm_demog, mm_cult])

## Split datasets into decades & save

In [ ]:
first_decade = combo_train_df[combo_train_df['publicationYear'] <= 1979]
second_decade = combo_train_df[(combo_train_df['publicationYear'] >= 1980) & (combo_train_df['publicationYear'] <= 1989) ]

third_decade = combo_train_df[(combo_train_df['publicationYear'] >= 1990) & (combo_train_df['publicationYear'] <= 1999) ]


fourth_decade = combo_train_df[(combo_train_df['publicationYear'] >= 2000) & (combo_train_df['publicationYear'] <= 2015) ]


first_decade.to_csv('first_decade.csv')
second_decade.to_csv('second_decade.csv')
third_decade.to_csv('third_decade.csv')
fourth_decade.to_csv('fourth_decade.csv')

## Preprocess text for each decade

### Remove surnames, use enchant library to filter out non-English words

In [ ]:
import pandas as pd
import ast
import pickle

In [ ]:
## remove surnames

my_file = open("surnames.txt", "r")
data = my_file.read()
surnames = data.split(",")
surnames = [i.replace("'", '').strip() for i in surnames]
my_file.close()

In [ ]:
! pip install pyenchant

In [ ]:
import enchant
valid_d = enchant.Dict("en_US") 

In [ ]:
def process_for_decade(decade_df, decade):
    """
    remove surnames, filter by enchant dictionary, and filter by word length
    """
    
    p = [ast.literal_eval(i) for i in decade_df['processed']]
    
    processed_again = []

    for i in tqdm(p):
        k = [j for j in i if j not in surnames]
        processed_again.append(k)
        
    processed_twice = []
    for i in tqdm(processed_again):
        k = [el for el in i if el.isalpha() and valid_d.check(el)]
        processed_twice.append(k)
    
    for k in processed_twice:
        processed_again2.append([i for i in k if len(i)>2])
    
    with open('processed_corp_enchant_' + decade +'.pkl', 'wb') as f:
        pickle.dump(processed_twice, f)
    
    return processed_twice

In [ ]:
processed_first_decade = process_for_decade(first_decade, 'first')
processed_second_decade = process_for_decade(second_decade, 'second')
processed_third_decade = process_for_decade(third_decade, 'third')
processed_fourth_decade = process_for_decade(fourth_decade, 'fourth')

## Train gensim phrased word2vec models

In [ ]:
from gensim.models.phrases import Phrases

In [ ]:
## build bigrams to create phased w2v models
from gensim.models.phrases import Phrases, Phraser
def build_phrases(sentences):
    phrases = Phrases(sentences, min_count = 5, threshold = 7, progress_per = 1000)
    return Phraser(phrases)

In [ ]:
bigram1 = build_phrases(processed_first_decade)
bigram2 = build_phrases(processed_second_decade)
bigram3 = build_phrases(processed_third_decade)
bigram4 = build_phrases(processed_fourth_decade)

In [ ]:
processed_bigrams1 = [bigram1[i] for i in processed_first_decade]
processed_bigrams2 = [bigram2[i] for i in processed_second_decade]
processed_bigrams3 = [bigram3[i] for i in processed_third_decade]
processed_bigrams4 = [bigram4[i] for i in processed_fourth_decade]

In [ ]:
processed_bigrams_final1 = []
processed_bigrams_final2 = []
processed_bigrams_final3 = []
processed_bigrams_final4 = []

## strip punctuations
for k in processed_bigrams1:
    processed_bigrams_final1.append([i.strip('!"“#$%&\'()*+,-./:;<=>?@[\\]^`{|}~…') for i in k if len(i)>2])

for k in processed_bigrams2:
    processed_bigrams_final2.append([i.strip('!"“#$%&\'()*+,-./:;<=>?@[\\]^`{|}~…') for i in k if len(i)>2])

for k in processed_bigrams3:
    processed_bigrams_final3.append([i.strip('!"“#$%&\'()*+,-./:;<=>?@[\\]^`{|}~…') for i in k if len(i)>2])

for k in processed_bigrams4:
    processed_bigrams_final4.append([i.strip('!"“#$%&\'()*+,-./:;<=>?@[\\]^`{|}~…') for i in k if len(i)>2])

In [ ]:
import multiprocessing
from sklearn import utils
cores = multiprocessing.cpu_count()
import gensim
from gensim.test.utils import get_tmpfile

In [ ]:
model_decade_1 = gensim.models.Word2Vec(processed_bigrams_final1, vector_size = 300, window = 10,
                                       min_count = 5, sg = 1, alpha = 0.05, epochs = 50, 
                                       batch_words = 10000, workers = cores, seed = 0, negative = 5,ns_exponent = 0.75)



fname = "word2vec_phrased_filtered_enchant_300d_1970_1979_2022_apr4.bin"
model_decade_1.save(fname)
print("Model Saved!")

In [ ]:
model_decade_2 = gensim.models.Word2Vec(processed_bigrams_final2, vector_size = 300, window = 10,
                                       min_count = 5, sg = 1, alpha = 0.05, epochs = 50, 
                                       batch_words = 10000, workers = cores, seed = 0, negative = 5,ns_exponent = 0.75)



fname = "word2vec_phrased_filtered_enchant_300d_1980_1989_2022_apr4.bin"
model_decade_2.save(fname)
print("Model Saved!")

In [ ]:
model_decade_3 = gensim.models.Word2Vec(processed_bigrams_final3, vector_size = 300, window = 10,
                                       min_count = 5, sg = 1, alpha = 0.05, epochs = 50, 
                                       batch_words = 10000, workers = cores, seed = 0, negative = 5,ns_exponent = 0.75)



fname = "word2vec_phrased_filtered_enchant_300d_1990_1999_2022_apr4.bin"
model_decade_3.save(fname)
print("Model Saved!")

In [ ]:
model_decade_4 = gensim.models.Word2Vec(processed_bigrams_final4, vector_size = 300, window = 10,
                                       min_count = 5, sg = 1, alpha = 0.05, epochs = 50, 
                                       batch_words = 10000, workers = cores, seed = 0, negative = 5,ns_exponent = 0.75)



fname = "word2vec_phrased_filtered_enchant_300d_2000_2016_2022_apr4.bin"
model_decade_4.save(fname)
print("Model Saved!")